<a href="https://colab.research.google.com/github/benjaminnigjeh/automated_data_mining/blob/main/spectral_search_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Fisher-py Module

In [ ]:
!apt-get update
!apt-get install -y mono-complete
!pip install fisher-py

#Install external libraries

In [ ]:
from fisher_py.data.business import Scan
from fisher_py import RawFile
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import pickle
import os
import re

# Three tier search engine

In [2]:
def engine(cast1, cast2):
    X = [cast1, cast2]
    return(round(cosine_similarity(X)[1][0], 2))

def matching(sample, databank, retention_window = 2, first_tier = 0.9, second_tier = 0.7, third_tier = 0.5):
    similarity = [0]*len(sample["cast spectra"])
    match_scan = [0]*len(sample["cast spectra"])
    match_sample = [0]*len(sample["cast spectra"])
    match_tier = [0]*len(sample["cast spectra"])
    ided_pfr = ['NaN']*len(sample["cast spectra"])
    ided_acc = ['NaN']*len(sample["cast spectra"])
    ided_uni = ['NaN']*len(sample["cast spectra"])
    uni_list = []
    retention = [0]*20
    mid = 0
    ret_index = 0

    for i in range(0, len(sample["cast spectra"])):
        for j in range(0, len(databank["cast spectra"])):
            if sample["m/z"][i] - 1 < databank["m/z"][j] < sample["m/z"][i] + 1:
                if engine(sample["cast spectra"][i], databank["cast spectra"][j]) > similarity[i]:
                    similarity[i] = engine(sample["cast spectra"][i], databank["cast spectra"][j])
                    match_scan[i] = databank["scan"][j]
                    match_sample[i] = databank['sample name'][j]
                    ret_index = j
                    if similarity[i] > first_tier:
                        match_tier[i] = 1
                        ided_pfr[i] = databank['PFR'][j]
                        ided_acc[i] = databank['Accession'][j]
                        ided_uni[i] = databank['Uniprot ID'][j]
                        uni_list.append(ided_uni[i])
                        retention.append(databank["retntion time"][ret_index])
                        mid = sum(retention[-20:]) / 20
        if similarity[i] < first_tier:
            similarity[i] = 0
            for k in [index for index, num in enumerate(databank["retntion time"]) if mid + retention_window > num > mid - retention_window]:
                if sample["m/z"][i] -1 < databank["m/z"][k] < sample["m/z"][i] + 1:
                    if engine(sample["cast spectra"][i], databank["cast spectra"][k]) > similarity[i]:
                        similarity[i] = engine(sample["cast spectra"][i], databank["cast spectra"][k])
                        if similarity[i] > third_tier:
                          if databank['Uniprot ID'][k] in uni_list:
                            match_tier[i] = 3
                            match_scan[i] = databank["scan"][k]
                            match_sample[i] = databank['sample name'][k]
                            ided_pfr[i] = databank['PFR'][k]
                            ided_acc[i] = databank['Accession'][k]
                            ided_uni[i] = databank['Uniprot ID'][k]
                        if similarity[i] > second_tier:
                          match_tier[i] = 2
                          match_scan[i] = databank["scan"][k]
                          match_sample[i] = databank['sample name'][k]
                          ided_pfr[i] = databank['PFR'][k]
                          ided_acc[i] = databank['Accession'][k]
                          ided_uni[i] = databank['Uniprot ID'][k]

    sample['matched_sample'] = match_sample
    sample['matched_scan'] =  match_scan
    sample['matched_similarity'] = similarity
    sample['matched_tier'] = match_tier
    sample['matched_PFR'] = ided_pfr
    sample['matched_Accession'] = ided_acc
    sample['matched_UniprotID'] = ided_uni
    return()